In [9]:
import pandas as pd
import os
import requests

def doi_to_csv():
    # Lee el archivo corpus.txt como una serie
    df_dois = pd.read_csv("corpus.txt", header=None, names=["DOIs"])
    dois = df_dois["DOIs"].tolist()

    documents_data = []
    authors_list = []
    author_columns = ['paperId', 'title', 'abstract', 'year', 'publicationDate', 'authorId', 'authorName']
    df_authors = pd.DataFrame(columns=author_columns)  # Inicializar el DataFrame df_authors

    for doi in dois:
        r = requests.get(f'https://api.semanticscholar.org/v1/paper/{doi}')
        if r.status_code == 200:
            data = r.json()

            document_info = [data.get('paperId', ''), data.get('title', ''), data.get('year', '')]
            documents_data.append(document_info)

            authors = data.get('authors', [])
            author_ids = [author.get('authorId', '') for author in authors]
            author_names = [author.get('name', '') for author in authors]
            authors_list.extend(author_names)

            author_info = [data.get('paperId', ''), data.get('title', ''), data.get('abstract', ''),
                           data.get('year', ''), data.get('publicationDate', ''), ','.join(author_ids),
                           ','.join(author_names)]
            df_authors.loc[len(df_authors)] = author_info

    if not os.path.exists('./csv'):
        os.makedirs('./csv')

    df_documents = pd.DataFrame(documents_data, columns=['file_name', 'title', 'publication_date'])

    df_documents.to_csv('./csv/documents.csv', index=False)
    df_authors.to_csv('./csv/authors.csv', index=False)
